# Nurse Staffing Recommendations

Analysis of the Center for Medicare & Medicaid Services Nurse Staffing
Dataset

Matthew Bain  
2024-03-22

I investigate nurse staffing data to provide informed recommendations to
a medical staffing organization.

\[…\]

## Imports

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from great_tables import GT
from pandas.plotting import scatter_matrix

from src.stylesheet import customize_plots
from src.inspection import make_df, display

## The dataset

We begin by exploring the data to get to know the features and patterns
on which we will base our analysis.

In [9]:
if 'data' not in locals():
    data = pd.read_csv(
        "../data/raw/PBJ_Daily_Nurse_Staffing_Q1_2024.zip",
        encoding='ISO-8859-1'
    )
else:
    print("data loaded.")

In [10]:
data.sample(5)

5 rows × 33 columns

In [11]:
# TODO: pivot on day

data_pivoted = data.pivot_table(
    index="STATE",
    columns="WorkDate",
    values="Hrs_RN",
    aggfunc='mean'
)

# Resetting the index for easier column access
# data_pivoted.reset_index(inplace=True)
data_pivoted

52 rows × 91 columns

In [24]:
data_pivoted.iloc[:, 1:]

52 rows × 90 columns

In [ ]:
(
    GT(data_pivoted, rowname_col="STATE")
    .fmt_nanoplot(
        columns=data_pivoted.columns[1:],
        reference_line="mean",
        reference_area=["min", "q1"]
    )
    .fmt_nanoplot(
        columns=data_pivoted.columns[1:],
        plot_type="bar",
        reference_line="max",
        reference_area=["max", "median"]
    )
)

In [80]:
data.describe().round(1)
# display(Markdown(data.describe().to_markdown()))

8 rows × 27 columns

In [81]:
attributes = ["Hrs_RN", "Hrs_LPN_ctr", "Hrs_CNA", "Hrs_NAtrn", "Hrs_MedAide"]
n = len(attributes)

fig, axs = plt.subplots(n, n, figsize=(8, 8))
scatter_matrix(
    data[attributes].sample(200),
    ax=axs, alpha=.7,
    hist_kwds=dict(bins=15, linewidth=0)
)
fig.align_ylabels(axs[:, 0])
fig.align_xlabels(axs[-1, :])
for ax in axs.flatten():
    ax.tick_params(axis='both', which='both', length=3.5)

# save_fig("scatter_matrix_plot")

plt.show()

In [125]:
from great_tables import GT

df = data.loc[150000:, [
    "STATE",
    "COUNTY_NAME", "COUNTY_FIPS",
    "CITY",
    "PROVNAME", "PROVNUM",
    # "MDScensus"
]].value_counts().reset_index()
GT(df.head(n=10))

## Some GT examples

In [91]:
from typing import Any
from IPython.display import display as ipy_display, HTML
import numpy as np

def display2(
    *args,
    globs: dict[str, Any] | None = None,
    bold: bool = True,
    width: str = "400px"  # Fixed width for each block
) -> None:
    """
    Display an informative representation of multiple objects side-by-side in Jupyter.

    Parameters
    ----------
    *args : tuple
        Tuple of expressions to evaluate and display.
    globs : dict[str, Any], default=None
        Global namespace, to give eval() access to nonlocals passed by name.
    bold : bool, default=True
        Option to enable/disable string styling.
    width : str, default="400px"
        Fixed width for each displayed block in the Jupyter notebook.

    Warnings
    --------
    This function uses `eval()` to render expressions it receives
    as strings. Access to variables in the global namespace is controlled
    by `globs`. Take care to only pass trusted expressions to the function.
    """

    if globs is None:
        globs = {}

    outputs = []
    for arg in args:
        name = f"<b>{arg}</b>" if bold else arg
        value = np.round(eval(arg, globs), 2)
        shape = np.shape(value)
        content = f"<div style='width:{width}; padding:10px; float:left;'><pre>{name}\n--- {repr(shape)} ---\n{repr(value)}</pre></div>"
        outputs.append(content)

    # Clearfix for layout
    clearfix = "<div style='clear: both;'></div>"

    # Display the HTML content in Jupyter
    html_output = ''.join(outputs) + clearfix
    ipy_display(HTML(html_output))

    return None

In [92]:
A = np.array([[1, 3], [2, 4]])
x = np.array([[0, 1]])

display2(
    "A", "x.T", "np.dot(A, x.T)", globs=globals(), bold=True, width="100px"
)

A 
--- (2, 2) ---
array([[1, 3],
 [2, 4]]) x.T 
--- (2, 1) ---
array([[0],
 [1]]) np.dot(A, x.T) 
--- (2, 1) ---
array([[3],
 [4]])

In [93]:
display2(
    "data['STATE'].value_counts()",
    "data['COUNTY_NAME'].value_counts()",
    "data['CITY'].value_counts()",
    "data['PROVNAME'].value_counts()",
    "data['MDScensus'].value_counts()",
    width="340px",
    globs=globals()
)

data['STATE'].value_counts() 
--- (52,) ---
STATE
TX 105924
CA 103740
OH 83538
FL 63245
IL 61425
PA 60515
NY 54691
IN 46501
MO 44226
MI 38402
NC 37310
IA 36036
GA 31941
NJ 31395
MA 31122
MN 30758
WI 29484
TN 27300
KS 27118
VA 26117
OK 25116
KY 24115
LA 23751
AL 20293
MD 20020
AR 19747
CO 19110
MS 18200
CT 17836
WA 17199
SC 16835
NE 16289
AZ 12649
OR 11557
WV 11102
UT 8918
SD 8554
ME 7371
ID 7280
ND 6734
RI 6643
NH 6643
NM 6006
NV 5824
MT 5369
DE 3913
HI 3731
VT 3094
WY 2912
DC 1547
AK 1365
PR 455
Name: count, dtype: int64 data['COUNTY_NAME'].value_counts() 
--- (1671,) ---
COUNTY_NAME
Los Angeles 33124
Cook 18564
Montgomery 16835
Jefferson 16471
Washington 12831
 ... 
Lanier 91
Twiggs 91
Vinton 91
Boundary 91
Refugio 91
Name: count, Length: 1671, dtype: int64 data['CITY'].value_counts() 
--- (5092,) ---
CITY
CHICAGO 7098
LOS ANGELES 6916
CINCINNATI 6279
HOUSTON 5824
SAN ANTONIO 5460
 ... 
LINCROFT 91
WYCKOFF 91
FLORHAM PARK 91
NEW BRUNSWICK 91
HORSESHOE BAY 91
Name: count, Length: 5092, dtype: int64 data['PROVNAME'].value_counts() 
--- (14441,) ---
PROVNAME
MILLER'S MERRY MANOR 1092
PARKVIEW CARE CENTER 455
LITTLE SISTERS OF THE POOR 455
HERITAGE MANOR 273
RIVERSIDE NURSING AND REHABILITATION CENTER 273
 ... 
MORGANFIELD NURSING AND REHABILITATION CENTER 91
CAMPBELLSVILLE NURSING AND REHABILITATION CENTER 91
FRANKLIN-SIMPSON NURSING AND REHABILITATION CENTER 91
GRAND HAVEN NURSING HOME 91
TIERRA ESTE NURSING AND REHABILITATION CENTER 91
Name: count, Length: 14441, dtype: int64 data['MDScensus'].value_counts() 
--- (635,) ---
MDScensus
56 15160
57 15116
58 15051
55 14830
53 14618
 ... 
622 1
629 1
730 1
634 1
617 1
Name: count, Length: 635, dtype: int64

In [94]:
data[["CY_Qtr", "WorkDate", "MDScensus"]]

1330966 rows × 3 columns

## SQL